In [1]:
from datasets import load_dataset
from collections import Counter

# ---------------------------------------------------------
# 1. Tải dữ liệu từ Hugging Face
# ---------------------------------------------------------
# Sử dụng hàm load_dataset để tải bộ dữ liệu CoNLL 2003 
print("Đang tải dữ liệu CoNLL 2003...")
dataset = load_dataset("conll2003", trust_remote_code=True)

# Dữ liệu trả về là DatasetDict chứa các split: train, validation, test [cite: 29]
print("Các tập dữ liệu:", dataset)

# ---------------------------------------------------------
# 2. Trích xuất câu và nhãn
# ---------------------------------------------------------
# Trích xuất các câu (tokens) và nhãn (ner_tags) từ tập train [cite: 32, 33]
train_sentences = dataset["train"]["tokens"]
train_tags_ids = dataset["train"]["ner_tags"]

# Lấy danh sách tên nhãn (string) từ features để ánh xạ từ số sang tên [cite: 35]
# Truy cập vào features["ner_tags"].feature.names
label_list = dataset["train"].features["ner_tags"].feature.names
print(f"\nDanh sách các nhãn (String labels): {label_list}")

# Chuyển đổi nhãn số sang dạng string (VD: 0 -> 'O', 1 -> 'B-PER', ...) [cite: 35]
# Bước này giúp kiểm tra dữ liệu, nhưng khi huấn luyện ta thường dùng ID.
# Ở đây ta tạo một danh sách các nhãn dạng string để minh họa theo yêu cầu.
train_tags_str = [[label_list[i] for i in sentence_tags] for sentence_tags in train_tags_ids]

# In thử 1 mẫu dữ liệu đầu tiên
print("\n--- Mẫu dữ liệu đầu tiên ---")
print("Tokens:", train_sentences[0])
print("Tags (ID):", train_tags_ids[0])
print("Tags (String):", train_tags_str[0])

# ---------------------------------------------------------
# 3. Xây dựng Từ điển (Vocabulary)
# ---------------------------------------------------------
# Tạo word_to_ix: Ánh xạ mỗi từ duy nhất sang một chỉ số index [cite: 38]
word_counts = Counter()
for sentence in train_sentences:
    word_counts.update(sentence)

# Thêm token đặc biệt <PAD> (đệm) và <UNK> (từ lạ) 
word_to_ix = {"<PAD>": 0, "<UNK>": 1}

# Duyệt qua các từ đã đếm được và thêm vào từ điển
for word in word_counts:
    if word not in word_to_ix:
        word_to_ix[word] = len(word_to_ix)

# Tạo tag_to_ix: Ánh xạ mỗi nhãn NER (dạng string) sang chỉ số nguyên 
# Ta dùng danh sách label_list đã lấy ở trên
tag_to_ix = {label: i for i, label in enumerate(label_list)}

# ---------------------------------------------------------
# Kết quả
# ---------------------------------------------------------
# In ra kích thước của hai từ điển [cite: 41]
print("\n--- Kích thước từ điển ---")
print(f"Kích thước word_to_ix (Vocab size): {len(word_to_ix)}")
print(f"Kích thước tag_to_ix (Num labels): {len(tag_to_ix)}")
print(f"Danh sách tag_to_ix: {tag_to_ix}")

Đang tải dữ liệu CoNLL 2003...


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Các tập dữ liệu: DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

Danh sách các nhãn (String labels): ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

--- Mẫu dữ liệu đầu tiên ---
Tokens: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
Tags (ID): [3, 0, 7, 0, 0, 0, 7, 0, 0]
Tags (String): ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

--- Kích thước từ điển ---
Kích thước word_to_ix (Vocab size): 23625
Kích thước tag_to_ix (Num labels): 9
Danh sách tag_to_ix: {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# ---------------------------------------------------------
# 1. Tạo lớp NERDataset
# ---------------------------------------------------------
class NERDataset(Dataset):
    # Kế thừa từ torch.utils.data.Dataset [cite: 44]
    def __init__(self, sentences, tags, word_to_ix, tag_to_ix):
        """
        Khởi tạo dataset [cite: 45]
        :param sentences: Danh sách các câu (dạng token)
        :param tags: Danh sách các chuỗi nhãn tương ứng (hoặc ID nhãn)
        :param word_to_ix: Từ điển ánh xạ từ -> index
        :param tag_to_ix: Từ điển ánh xạ nhãn -> index
        """
        self.sentences = sentences
        self.tags = tags
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix

    def __len__(self):
        # Trả về tổng số câu trong bộ dữ liệu [cite: 46]
        return len(self.sentences)

    def __getitem__(self, idx):
        # Lấy câu và nhãn tại vị trí index
        tokens = self.sentences[idx]
        label_data = self.tags[idx] # Ở Task 1 ta đã có nhãn dạng số hoặc string

        # Chuyển đổi token thành index (dùng <UNK> nếu từ không có trong từ điển) 
        # word_to_ix.get(token, word_to_ix["<UNK>"])
        encoded_sents = [self.word_to_ix.get(token, self.word_to_ix["<UNK>"]) for token in tokens]
        
        # Xử lý nhãn:
        # Nếu label_data là list các chuỗi (như yêu cầu slide [cite: 45]), ta map qua tag_to_ix.
        # Nếu label_data đã là list các số nguyên (từ dataset gốc), ta dùng luôn.
        if isinstance(label_data[0], str):
             encoded_tags = [self.tag_to_ix[tag] for tag in label_data]
        else:
             encoded_tags = label_data

        # Trả về cặp tensor (sentence_indices, tag_indices) [cite: 47]
        return torch.tensor(encoded_sents, dtype=torch.long), torch.tensor(encoded_tags, dtype=torch.long)

# ---------------------------------------------------------
# 2. Viết hàm collate_fn để đệm (padding) dữ liệu
# ---------------------------------------------------------
def collate_fn(batch):
    """
    Hàm này được gọi bởi DataLoader để gộp các mẫu đơn lẻ thành một batch.
    Cần pad các câu về cùng độ dài của câu dài nhất trong batch[cite: 52].
    """
    # Tách batch (list các tuples) thành list câu và list nhãn riêng biệt
    sentences, tags = zip(*batch)
    
    # Lấy giá trị padding cho câu là index của <PAD> (thường là 0) 
    pad_val_word = word_to_ix.get("<PAD>", 0)
    
    # Lấy giá trị padding cho nhãn (ví dụ: -1) để bỏ qua khi tính loss sau này 
    pad_val_tag = -1 
    
    # Sử dụng pad_sequence với batch_first=True 
    padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=pad_val_word)
    padded_tags = pad_sequence(tags, batch_first=True, padding_value=pad_val_tag)
    
    return padded_sentences, padded_tags

# ---------------------------------------------------------
# 3. Khởi tạo DataLoader
# ---------------------------------------------------------
# Chuẩn bị dữ liệu đầu vào (Lấy từ biến dataset ở Task 1)
# Lưu ý: dataset["train"]["ner_tags"] là số nguyên, dataset["train"]["tokens"] là list từ.
# Để khớp với slide yêu cầu input là "danh sách chuỗi nhãn"[cite: 45], 
# ta có thể dùng biến train_tags_str đã tạo ở Task 1, hoặc dùng thẳng ID gốc cho tiện.
# Dưới đây mình dùng dữ liệu gốc từ HuggingFace cho tối ưu.

train_sentences_data = dataset["train"]["tokens"]
train_tags_data = dataset["train"]["ner_tags"]

val_sentences_data = dataset["validation"]["tokens"]
val_tags_data = dataset["validation"]["ner_tags"]

# Khởi tạo dataset cho tập Train và Validation
train_ds = NERDataset(train_sentences_data, train_tags_data, word_to_ix, tag_to_ix)
val_ds = NERDataset(val_sentences_data, val_tags_data, word_to_ix, tag_to_ix)

# Khởi tạo DataLoader [cite: 50, 51]
BATCH_SIZE = 32 # Bạn có thể chỉnh batch size tùy ý

train_loader = DataLoader(
    train_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=True,           # Train nên shuffle dữ liệu
    collate_fn=collate_fn   # Sử dụng hàm padding tùy chỉnh [cite: 52]
)

val_loader = DataLoader(
    val_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=False,          # Validation không cần shuffle
    collate_fn=collate_fn
)

# ---------------------------------------------------------
# Kiểm tra thử 1 batch
# ---------------------------------------------------------
print("--- Kiểm tra DataLoader ---")
# Lấy thử 1 batch từ train_loader
sample_sents, sample_tags = next(iter(train_loader))

print(f"Shape của batch câu (Batch size, Max Seq Len): {sample_sents.shape}")
print(f"Shape của batch nhãn: {sample_tags.shape}")
print(f"Ví dụ câu đầu tiên trong batch (đã padding): \n{sample_sents[0]}")
print(f"Ví dụ nhãn đầu tiên trong batch (đã padding -1): \n{sample_tags[0]}")

--- Kiểm tra DataLoader ---
Shape của batch câu (Batch size, Max Seq Len): torch.Size([32, 43])
Shape của batch nhãn: torch.Size([32, 43])
Ví dụ câu đầu tiên trong batch (đã padding): 
tensor([2034, 7502, 7503,  131, 1825,  133, 7507, 6864,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0])
Ví dụ nhãn đầu tiên trong batch (đã padding -1): 
tensor([ 0,  1,  2,  0,  5,  0,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1])


In [3]:
import torch.nn as nn

class SimpleRNNForNER(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        """
        Khởi tạo mô hình RNN cho bài toán NER.
        
        Args:
            vocab_size (int): Kích thước từ điển (số lượng từ unique).
            embedding_dim (int): Số chiều của vector biểu diễn từ (Embedding).
            hidden_dim (int): Số chiều của trạng thái ẩn trong RNN.
            output_size (int): Số lượng nhãn NER (số lớp đầu ra).
        """
        super(SimpleRNNForNER, self).__init__()
        
        # 1. nn.Embedding: Chuyển đổi chỉ số của từ thành vector 
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # 2. nn.RNN: Xử lý chuỗi vector embedding [cite: 58]
        # batch_first=True: Input shape sẽ là (batch_size, seq_len, features)
        # Bạn có thể thay nn.RNN bằng nn.LSTM hoặc nn.GRU để mô hình học tốt hơn [cite: 58]
        self.rnn = nn.RNN(input_size=embedding_dim, 
                          hidden_size=hidden_dim, 
                          batch_first=True)
        
        # 3. nn.Linear: Ánh xạ output của RNN sang không gian nhãn để dự đoán 
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        """
        Lan truyền xuôi (Forward pass)
        x shape: (batch_size, seq_len) - chứa các chỉ số từ (indices)
        """
        # B1: Qua lớp Embedding
        # shape: (batch_size, seq_len, embedding_dim)
        embedded = self.embedding(x)
        
        # B2: Qua lớp RNN
        # rnn_out shape: (batch_size, seq_len, hidden_dim)
        # h_n là trạng thái ẩn cuối cùng (không cần dùng ở đây)
        rnn_out, h_n = self.rnn(embedded)
        
        # B3: Qua lớp Linear (Fully Connected)
        # Áp dụng Linear cho từng token trong chuỗi
        # logits shape: (batch_size, seq_len, output_size)
        logits = self.fc(rnn_out)
        
        return logits

# ---------------------------------------------------------
# Khởi tạo mô hình với các tham số phù hợp [cite: 62]
# ---------------------------------------------------------

# Các siêu tham số (Hyperparameters)
VOCAB_SIZE = len(word_to_ix)       # Kích thước từ điển đã tạo ở Task 1
EMBEDDING_DIM = 100                # Chiều dài vector embedding (thường là 100, 200, 300)
HIDDEN_DIM = 128                   # Kích thước hidden state của RNN
OUTPUT_SIZE = len(tag_to_ix)       # Số lượng nhãn đầu ra (số class)

# Kiểm tra xem có GPU không để đẩy mô hình vào
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Tạo instance của mô hình
model = SimpleRNNForNER(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_SIZE)
model = model.to(device) # Đưa mô hình vào GPU/CPU

print(model)

Sử dụng thiết bị: cuda
SimpleRNNForNER(
  (embedding): Embedding(23625, 100)
  (rnn): RNN(100, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=9, bias=True)
)


In [4]:
import torch.optim as optim

# ---------------------------------------------------------
# 1. Khởi tạo Optimizer và Loss Function
# ---------------------------------------------------------
# Sử dụng Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Sử dụng CrossEntropyLoss
# Quan trọng: ignore_index=-1 để bỏ qua padding (như đã định nghĩa ở collate_fn)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

# ---------------------------------------------------------
# 2. Vòng lặp huấn luyện
# ---------------------------------------------------------
EPOCHS = 5  # Số lượng epoch (vòng lặp qua toàn bộ dữ liệu)

print(f"Bắt đầu huấn luyện trên {device}...")

for epoch in range(EPOCHS):
    model.train()  # Đặt mô hình ở chế độ huấn luyện
    total_loss = 0
    
    for batch_idx, (sentences, tags) in enumerate(train_loader):
        # Chuyển dữ liệu sang thiết bị (GPU/CPU)
        sentences = sentences.to(device)
        tags = tags.to(device)
        
        # --- 5 Bước huấn luyện kinh điển ---
        
        # 1. Xóa gradient cũ
        optimizer.zero_grad()
        
        # 2. Forward pass: Tính toán đầu ra
        # output shape: (batch_size, seq_len, output_size)
        predictions = model(sentences)
        
        # 3. Tính Loss
        # CrossEntropyLoss yêu cầu input: (N, C) và target: (N)
        # Ta cần làm phẳng (flatten) tensor để tính loss cho tất cả token
        predictions = predictions.view(-1, OUTPUT_SIZE) 
        tags = tags.view(-1)
        
        loss = criterion(predictions, tags)
        
        # 4. Backward pass: Tính gradient
        loss.backward()
        
        # 5. Cập nhật trọng số
        optimizer.step()
        
        total_loss += loss.item()
        
    # Tính loss trung bình của epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss trung bình: {avg_loss:.4f}")

print("Huấn luyện hoàn tất!")

Bắt đầu huấn luyện trên cuda...
Epoch 1/5 | Loss trung bình: 0.3697
Epoch 2/5 | Loss trung bình: 0.1077
Epoch 3/5 | Loss trung bình: 0.0539
Epoch 4/5 | Loss trung bình: 0.0377
Epoch 5/5 | Loss trung bình: 0.0309
Huấn luyện hoàn tất!


In [5]:
def evaluate(model, data_loader):
    model.eval()  # Đặt mô hình ở chế độ đánh giá [cite: 77]
    
    total_correct = 0
    total_count = 0
    
    with torch.no_grad():  # Tắt tính toán gradient [cite: 78]
        for sentences, tags in data_loader:
            sentences = sentences.to(device)
            tags = tags.to(device)
            
            # Lấy dự đoán từ mô hình
            outputs = model(sentences)
            
            # Lấy nhãn có xác suất cao nhất (argmax) trên chiều cuối cùng [cite: 80]
            # predictions shape: (batch_size, seq_len)
            predictions = torch.argmax(outputs, dim=2)
            
            # Tính toán độ chính xác
            # Chỉ tính trên các token không phải là padding (tag != -1) [cite: 82]
            mask = (tags != -1)
            
            # So sánh dự đoán với nhãn thật tại các vị trí hợp lệ
            correct_predictions = (predictions == tags) & mask
            
            total_correct += correct_predictions.sum().item()
            total_count += mask.sum().item()
            
    return total_correct / total_count

# Gọi hàm đánh giá và in kết quả [cite: 86]
val_accuracy = evaluate(model, val_loader)
print(f"Độ chính xác trên tập validation: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")

Độ chính xác trên tập validation: 0.9504 (95.04%)


In [6]:
def predict_sentence(sentence):
    model.eval()
    
    # 1. Tiền xử lý: Tách từ (đơn giản bằng split)
    tokens = sentence.split()
    
    # 2. Chuyển từ sang index
    # Dùng <UNK> nếu từ không có trong từ điển
    input_ids = [word_to_ix.get(word, word_to_ix["<UNK>"]) for word in tokens]
    
    # 3. Tạo tensor và đưa vào device (thêm chiều batch_size = 1)
    input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)
    
    # 4. Dự đoán
    with torch.no_grad():
        output = model(input_tensor)
        # Lấy index của nhãn dự đoán
        predicted_indices = torch.argmax(output, dim=2).squeeze().tolist()
    
    # 5. Ánh xạ ngược từ index sang nhãn string
    # Chúng ta cần list nhãn để tra cứu (label_list đã lấy ở Task 1)
    # Nếu predicted_indices là một số int (trường hợp câu 1 từ), chuyển thành list
    if isinstance(predicted_indices, int):
        predicted_indices = [predicted_indices]
        
    predicted_labels = [label_list[idx] for idx in predicted_indices]
    
    # 6. In kết quả
    print(f"\nCâu: \"{sentence}\"")
    print(f"{'Token':<15} {'Nhãn dự đoán':<15}")
    print("-" * 30)
    for token, label in zip(tokens, predicted_labels):
        print(f"{token:<15} {label:<15}")

# --- Ví dụ thực nghiệm theo yêu cầu tài liệu [cite: 96] ---
test_sentence = "VNU University is located in Hanoi"
predict_sentence(test_sentence)

# Thử thêm một câu khác từ dataset mẫu
predict_sentence("Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .")


Câu: "VNU University is located in Hanoi"
Token           Nhãn dự đoán   
------------------------------
VNU             B-ORG          
University      I-ORG          
is              O              
located         O              
in              O              
Hanoi           O              

Câu: "Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer ."
Token           Nhãn dự đoán   
------------------------------
Germany         B-LOC          
's              O              
representative  O              
to              O              
the             O              
European        B-ORG          
Union           I-ORG          
's              O              
veterinary      O              
committee       O              
Werner          B-PER          
Zwingmann       I-PER          
said            O              
o